# Altair with simple, sample data

- **[Altair documentation](https://altair-viz.github.io/index.html)**


- [Vega-Lite site](https://vega.github.io/vega-lite/)
- [Vega-Lite documentation](https://vega.github.io/vega-lite/docs/)
- [Vega-Lite 2.0 Medium article](https://medium.com/@uwdata/introducing-vega-lite-2-0-de6661c12d58)
- [Vega-Lite 2.0 OpenVisConf 2017 talk](https://www.youtube.com/watch?v=9uaHRWj04D4)
- [About the Vega project](https://vega.github.io/vega/about/)

#### This is just a made-up data set inspired by a [Nature Methods article](https://www.nature.com/articles/nmeth.2807)

In [200]:
import pandas as pd
import altair as alt

## Load in sample data

We'll load all data into a Panda DataFrame. A DataFrame is just a special data structure that is meant for "tablular data", which is like a spreadsheet. DataFrames also have build-in functions that can modify and display the data.

This pretend data set has values for five items in five categories. It gives us a chance to play around with various visual representations. **The best choice depends on which comparisons are most important to the story you're trying to tell!**

*Please excuse the numbers before the names of both items and categories. The data patterns are more clear with the original sorting, when instead of vegetables they had Item 1, Item 2... and instead of characteristics they had Category 1, Category 2. Since the default sorting in Altair (and almost all software) is alphabetical, and it getting it to not sort would have introduced extra code, I decided on this non-ideal compromise to keep the sort order with my new descriptor names.*


In [201]:
df = pd.read_csv('data/NatureVegSurveyScores.tsv', sep='\t', encoding='utf-8')
df.head()

,respID,veg,trait,response
0,r0,1_Corn,1_Green,1
1,r1,1_Corn,1_Green,2
2,r2,1_Corn,1_Green,4
3,r3,1_Corn,1_Green,1
4,r4,1_Corn,1_Green,1


### Data is already "tidy"

Hadley Wickham's [original tidy data paper](http://vita.had.co.nz/papers/tidy-data.html)


### Original survey data looked like this, in "wide" form

The original data from the Nature Methods article had a very generic five items in five categories. Here's I've tried to make it more concrete: **Let's say we have five different vegetables, and the values are the number of people that thought these vegetables were very Green, Yellow, Cheap, Tasty or Gross.**


*Note: this data is really just synthetic – we made it up to show how different visual representations change how we see the data patterns.*

**Fifty people responded to the survey questions like this:**

```
"Corn is very Green"

(5) Strongly agree
(4) Agree
(3) Neutral
(2) Disagree
(1) Strongly disagree
```

and our data consists of the numeric versions of the reponses.

In [202]:
df.pivot_table(index='respID', columns=['veg','trait'], values='response').head()

veg     1_Corn                                  2_Squash                   \
trait  1_Green 2_Yellow 3_Cheap 4_Tasty 5_Gross  1_Green 2_Yellow 3_Cheap   
respID                                                                      
r0           1        5       3       5       1        1        4       5   
r1           2        5       3       4       1        1        3       1   
r10          1        5       3       5       1        1        5       2   
r11          3        5       2       5       4        2        5       2   
r12          5        5       2       5       1        2        5       2   

veg                     ... 4_Green beans                                   \
trait  4_Tasty 5_Gross  ...       1_Green 2_Yellow 3_Cheap 4_Tasty 5_Gross   
respID                  ...                                                  
r0           3       1  ...             4        5       3       1       1   
r1           3       2  ...             3        2       2       4       1   
r10          3       2  ...             3        3       3       1       1   
r11          4       3  ...             4        4       4       4       1   
r12          4       4  ...             3        2       5       2       1   

veg     5_Peas                                   
trait  1_Green 2_Yellow 3_Cheap 4_Tasty 5_Gross  
respID                                           
r0           2        2       4       2       1  
r1           5        1       3       5       1  
r10          3        3       4       4       1  
r11          1        3       3       4       1  
r12          2        1       1       1       2  

[5 rows x 25 columns]

## Table to visualize & summarize the data

**Altair isn't great for making tables, so we'll just use a Pandas pivot table**

- The table is compact
- it communicates values very precisely
- but it's hard for us to look at a bunch of numbers, take them in, and see patterns in them
- it also doesn't tell any particular story

In [203]:
(df.pivot_table(index='veg',
               columns='trait',
               values='response',
               aggfunc='mean')
 .style.format(precision=1)
#.background_gradient(vmin=1, vmax=5)
)

trait,1_Green,2_Yellow,3_Cheap,4_Tasty,5_Gross
veg,,,,,
1_Corn,1.8,4.7,3.3,4.9,1.9
2_Squash,2.0,4.5,3.2,2.7,2.4
3_Brussel sprouts,2.3,3.7,3.1,1.5,3.5
4_Green beans,3.6,3.2,3.0,2.1,1.9
5_Peas,4.0,1.7,2.9,3.5,1.3


## Method chaining

In Altair it's easy to construct our visualization by **chaining together, with a dot between** a "declaration" of our visualization following the pattern:

- `alt` – calls the Altair module through it's abbreviated name stated above
- `Chart()` – feed in the Pandas DataFrame our data values come from
- `mark_xxxx()` – sets the [mark type](https://altair-viz.github.io/user_guide/marks.html) to use – here a rectangle
- `encode()` – specified the "[encoding channels](https://altair-viz.github.io/user_guide/encoding.html#encoding-channels)" for this visualization, things like x (horizontal axis), y (vertical axis), color, tooltip, shape, size, etc.
- `transform_xxxx()` – [data transformations](https://altair-viz.github.io/user_guide/transform.html) like filter, calculate, aggregate, lookup, etc. 

## Heatmap

A heatmap is a very compact visual representation of the data, very similar to the original table, where rectangles are colored by the values in each cell. *Note that `mark_rect()` makes a rectangle that will always fill the cell, which is perfect for making heatmaps. A `mark_square()` is a square which can have size variation.*

We're not really good at quantitatively comparing color values, though, so this isn't a great representation if you want people to accurately detect the numerical patterns. Also, note that Cheap values aren't distinguishable.

**Let's practice typing the specification! Delete and retype the code.**

Always start with `alt.Chart().mark_rect().encode()` and then fill in the DataFrame and encoding

```
alt.Chart(df).mark_rect().encode(
    x = 'trait',
    y = 'veg',
    color = 'mean(response)'
)
```

- The heatmap is compact and eye-catching
- it tends to show blocks of light and dark, which isn't great for this data
- it also doesn't tell any particular story here

In [204]:
alt.Chart(df).mark_rect().encode(
    x = 'trait',
    y = 'veg',
    color = 'mean(response)'
)

alt.Chart(...)

## Circle area encoding

Let's start generating other alternative visual encodings for the data that will be better suited for particular comparisons we're trying to make easy for the audience.

- **Note that now we're specifying the variable types**, in this case so Altair will give us a categorical (nominal) color scheme instead of ordinal.
- **It's a good idea to always specify the data types!**

[Encoding data types documentation](https://altair-viz.github.io/user_guide/encoding.html#encoding-data-types)

|Data Type|Shorthand Code|Description
|---|---|---
|quantitative|Q|a continuous real-valued quantity
|ordinal|O|a discrete ordered quantity
|nominal|N|a discrete unordered category
|temporal|T|a time or date value
|geojson|G|a geographic shape

**Try switching the color variable (encoding data) type from "nominal" to "ordinal" and see what changes.**

- The color makes us see this in columns, because we group items with similar visual properties
- that starts telling the story of how the traits vary across the vegetables 
- but we're not great at quantitatively comparing areas
- note that these patterns wouldn't be clear without this particular sorting

In [205]:
alt.Chart(df).mark_circle().encode(
    x = 'trait:N',
    y = 'veg:N',
    color = 'trait:N',
    size = 'mean(response):Q'
)

alt.Chart(...)

### Changing plot size

Also showing you can use a square instead of a circle for a similar effect

In [206]:
alt.Chart(df).mark_square().encode(
    x = 'trait:N',
    y = 'veg:N',
    color = 'trait:N',
    size = 'mean(response):Q'
).properties(
    width = 200,
    height = 200
)

alt.Chart(...)

---

## EXERCISE 1

Try making a bar chart with:

- a **horizontal bar for each vegetable** (with labels on the left), 
- each bar length representing the **mean numeric response** within the vegetable

![goal horiz_bars](images/veghorizbars.png)

```
alt.Chart(----).mark_bar().encode(
    x = '----:-',
    y = '----:-'
)
```

- Use the code above as a hint, but **type instead of copy/paste** 
- Replace the dashes with correct code

### Line plot

Another way to show the trends within the traits across the vegetables would be to make a line plot.

**It's not usually a great idea to connect categorical variables with lines, since there is nothing between the vegetables**, but let's try it here to guide the eye.


In [207]:
alt.Chart(df).mark_line().encode(
    x = 'mean(response):Q',
    y = 'veg:N',
    color = 'trait:N'
)

alt.Chart(...)

### Layering charts

[Layered and multi-view charts documentation]()

Let's put a dot on the lines to make it clear where the data points are. 

- **Altair let's us use a `+` to layer individual charts and match their axes:** `lines + dots`
- An alternative syntax is: `alt.layer(lines, dots)`
- This is such a common need, to add dots to lines, that there is a shortcut without layering: `.mark_line(point=True)`

In [208]:
dots = alt.Chart(df).mark_circle().encode(
    x = 'mean(response):Q',
    y = 'veg:N',
    color = 'trait:N'
)

lines = alt.Chart(df).mark_line().encode(
    x = 'mean(response):Q',
    y = 'veg:N',
    color = 'trait:N'
)

lines + dots

alt.LayerChart(...)

## Changing the size of the layered plot

First, the aspect ratio of this plot makes it hard to follow some of the steep lines, so let's change the size and re-configure the axis grids at the same time. *We can configure the layered plot by putting them in parentheses and chaining on the properties*


In [209]:
dots = alt.Chart(df).mark_circle().encode(
    x = 'mean(response):Q',
    y = 'veg:N',
    color = 'trait:N'
)

lines = alt.Chart(df).mark_line().encode(
    x = 'mean(response):Q',
    y = 'veg:N',
    color = 'trait:N'
)

(lines + dots).properties(
    width = 250,
    height = 200
)

alt.LayerChart(...)

## Reducing repeated elements with a base plot

We can see in the example above that the lines and dots plots are almost exactly the same. Altair lets us use that and just add the differences.

- We'll define a base plot with all of the common elements, and 
- then add the differences to each right before layering

In [210]:
base = alt.Chart(df).encode(
    x = 'mean(response):Q',
    y = 'veg:N',
    color = 'trait:N'
).properties(
    width = 250,
    height = 200
)

base.mark_line() + base.mark_circle()

alt.LayerChart(...)

### Layering to add labels

`.mark_text()` is used for labels

In [211]:
base = alt.Chart(df).encode(
    y = 'veg:N',
    x = 'mean(response):Q'
)

alt.layer(
    base.mark_bar(color = 'orange'),
    bars.mark_text().encode(
        text = 'mean(response):Q'
    )
)

alt.LayerChart(...)

#### With some extra formatting

In [212]:
base = alt.Chart(df).encode(
    y = 'veg:N',
    x = alt.X('mean(response):Q').scale(domain=[0,4])
)

alt.layer(
    base.mark_bar(color = '#C84E00'),
    bars.mark_text(dx = 5, align='left').encode(
        text = alt.Text('mean(response):Q').format('.1f')
    )
)

alt.LayerChart(...)

## Faceted plots

We can make what are sometimes called "small multiples" in Altair using `facet()` to specify that facets, or unique values, of a categorical variable should be split off and arranged along either rows or columns of the overall visualization. **Visuals shown within each facet are only the subset of the data corresponding to that category!**

*Note that we could alternatively specified `column = 'trait:N'` right within the encoding instead of needing the `.facet()` section. The advantage of using `.facet()` is that you can make faceted views of more complicated charts.*

**Try changing the column facet to a row facet.** (You'll probably want to modify the height and width properties.) I prefer the row-faceted version beause I have an easier time comparing across vegetables.

*Note that since the traits are directly labeled we don't need a color legend.*

In [213]:
alt.Chart(df).mark_bar().encode(
    x = 'mean(response):Q',
    y = 'veg:N',
    color = alt.Color('trait:N').legend(None)
).properties(
    width = 80,
    height = 120
).facet(
    column='trait:N'
)

alt.FacetChart(...)

### Facets to see distributions of all the ratings

Using faceting we can even see bar charts of the counts of the numbers of people scoring 1-5 on all of the combinations quite easily!

In [214]:
alt.Chart(df).mark_bar().encode(
    x = alt.X('distinct(respID)').title('count'),
    y = 'response',
    row = 'veg',
    column = 'trait'
).properties(
    height=40, 
    width=40
).configure_headerRow(
    labelAngle = 0,
    labelAnchor = 'middle',
    labelAlign = 'left'
)

alt.Chart(...)

---

## Comparing within vegetable traits

### Stacked bars

Stacked bars might be a reasonable visual representation if part of the main story is the overall sums within vegetables.

- To get a stacked bar chart we just need to color by the trait
- *I'm not sure it makes sense to stack bars of the mean responses in each trait category, which is basically summing the mean responses...*

In [215]:
alt.Chart(df).mark_bar().encode(
    x = 'mean(response):Q',
    y = 'veg:N',
    color = 'trait:N'
)

alt.Chart(...)

### Labels on stacked bars are a bit tricky

https://altair-viz.github.io/gallery/stacked_bar_chart_with_text.html

In [216]:
bars = alt.Chart(df).mark_bar().encode(
    x = alt.X('mean(response):Q', stack='zero'),
    y = 'veg:N',
    color = 'trait:N'
)

labels = bars.mark_text(dx=-10).encode(
    text=alt.Text('mean(response):Q').format('.1f'),
    detail = 'trait:N',
    color = alt.value('white')
)

bars + labels

alt.LayerChart(...)

### Pie charts

In [217]:
alt.Chart(df).mark_arc().encode(
    theta = 'mean(response):Q',
    color = 'trait:N'
).properties(
    width = 80,
    height = 80
).facet(
    column='veg:N'
)

alt.FacetChart(...)

### Faceted labeled pie charts

For the labels I want to grab a substring of the original "trait" data entry

[Vega-lite expressions](https://vega.github.io/vega/docs/expressions/)

In [218]:
from altair import expr

In [219]:
base = alt.Chart(df).encode(
    alt.Theta("mean(response):Q").stack(True),
    alt.Color('trait:N').legend(None)
).properties(
    width = 80,
    height = 80
)

pie = base.mark_arc()

text = base.mark_text(radius=55, size=9).encode(
    text = "characteristic:N",
    detail = 'trait:N'
).transform_calculate(
    characteristic = expr.substring(alt.datum.trait, 2)
)

(pie + text).facet(column='veg:N')

alt.FacetChart(...)

### Configuration: Dot plot with horizontal grid lines

I find configuration of things like font sizes, colors, and placement to be difficult and annoying in almost every piece of visualization software (Excel, Tableau, etc.), but it's often necessary to customize these settings to tell your story effectively!

**[Top-level configuration docs](https://altair-viz.github.io/user_guide/configuration.html)** *(I find it quite difficult to find what I need in this documentation, even though it's very complete.)*

- Default is grid lines on continuous scale axes and not on nominal or ordinal, but I want lines along each item to help guide the eye and associate the dots with the vegetables, even though they're colored by traits.
- We also could have specified the circle size and opacity in the `encode()` section with `size = alt.value(150), opacity = alt.value(1.0)`
- Titles are a little strange to specify. 
- If you want multiple lines in either the title or subtitle, put the lines as a list of strings: `["line1","line2]`

In [220]:
alt.Chart(
    df, 
    title = alt.TitleParams(
        "Corn and peas have the strongest characteristics",
        subtitle = ["Corn is very yellow and tasty – Peas are very green, but not as tasty", "Squash is also very yellow"]
    )
).mark_circle(
    size = 150,
    opacity = 1.0,
).encode(
    x = 'mean(response):Q',
    y = 'veg:N',
    color = 'trait:N'
).properties(
    width = 400,
    height = 150,
).configure_axisX(
    grid=False
).configure_axisY(
    grid=True
).configure_axis(
    titleFontSize = 14,
    labelFontSize = 12
).configure_legend(
    titleFontSize = 14,
    labelFontSize = 12
).configure_title(
    fontSize = 20,
    anchor = 'start',
    offset = 15,
    color = "#333333",
    subtitleColor = "#666666"
)

alt.Chart(...)

### Faceted bars

If we want people to be able to make comparisons across traits within each vegetable, say whether Yellow-ness or Tasty-ness is larger for Corn, we need to give them a common baseline.

- Side-by-side grouped bars (column facets) works okay, but a bit hard to compare across vegetables

In [221]:
alt.Chart(df).mark_bar().encode(
    x = 'trait:N',
    y = 'mean(response):Q',
    color = alt.Color('trait:N').legend(None)
).properties(
    width = 100,
    height = 100
).facet(
    column='veg:N'
)

alt.FacetChart(...)

### Vertical (row) faceting makes some comparisons across vegetables easier

- While this is visually easier to look at
- the lack of a common baseline makes it harder to compare quantitatively across vegetables

In [222]:
alt.Chart(df).mark_bar().encode(
    x = 'trait:N',
    y = alt.Y('mean(response):Q').title('Avg response'),
    color = alt.Color('trait:N').legend(None)
).properties(
    width = 140,
    height = 70
).facet(
    row='veg:N'
)

alt.FacetChart(...)

---

## EXERCISE 2

Layer (superimpose) two visualizations:

1. Vertical bar chart showing
    - traits across the horizontal (bottom) axis
    - mean response (across all the vegetables) going up the vertical axis (bar height)
    - `color = alt.value('lightgray')`
1. Point plot of all individual response
    - use `mark_point()`
    - traits again along the horizontal axis
    - mean response up the vertical axis
    - color by trait
    
<!-- ![goal bars points](images/vegbarspoints.png) -->
    
*Point plot should be on top of the bar chart!*

In [223]:
base = alt.Chart(df).encode(x = 'trait:N')

bars = base.mark_bar().encode(
    y = 'mean(response):Q',
    color = alt.value('lightgray')
)

errors = base.mark_errorbar(extent='stdev').encode(
    y = 'response:Q',
    color = alt.Color('trait:N').legend(None),
    strokeWidth=alt.value(5)
)

(bars + errors).properties(width=200)

alt.LayerChart(...)

### Jitter

- By default the `yOffset` scales the random jitter to the full width of the band set for each categorical step in the y-axis
- Setting a scale domain on that offset channel tells Altair to not just read the possible spread of values from the data, but force it to this specified spread. This tricks the system into placing the jittered points away from the edges of the band for each y-value
- *This is kind of a weird setup because you're adding a new "jitter" column to the data, but since each point aggregates multiple rows, you also have to aggregate the jitter variable or the visualization shows the finest level of detail, which is a single row!*

In [224]:
alt.Chart(df).mark_point(
    opacity = 0.6, 
    strokeWidth = 3, 
    filled = False,
    size = 40,
    shape = 'circle'
).encode(
    x = 'mean(response):Q',
    y = alt.Y('rend:N').title('First digit in respID'),
    yOffset = alt.YOffset('mean(jitter):Q').scale(domain=[0.3,0.7]),
    color = alt.Color('rend:N').legend(None),
    detail = 'respID:N',
    tooltip = ['mean(response):Q','mean(jitter):Q']
).transform_calculate(
    jitter = 'random()',
    rend = expr.slice(alt.datum.respID,1,2)
).configure_axisY(
    titlePadding = 10,
    grid = True
)

alt.Chart(...)

---

# Vega-Lite specification

What Altair really produces is a Vega-Lite JSON declarative specification for building the visualization, and JupyterLab has Vega and Vega-Lite built in for rendering. This is a nice separation of concerns, where Altair just needs to know how to make JSON, and the renderer knows how to actually create the visuals!

We can see the specification behind each visualization by using `.to_json()` or `.to_dict()`. Here we'll use the latter, because the printout is more compact, but feel free to try the former.

**Note that all of the data is included in the JSON!!**

- [Medium article on Vega-Lite adoption](https://medium.com/@robin.linacre/why-im-backing-vega-lite-as-our-default-tool-for-data-visualisation-51c20970df39)

### Making some smaller data to demonstrate Vega-Lite JSON

In [225]:
df_temp = pd.DataFrame({'a':[1,2,0,2], 'b':[3,2,1,4]}).melt()
df_temp

,variable,value
0,a,1
1,a,2
2,a,0
3,a,2
4,b,3
5,b,2
6,b,1
7,b,4


In [226]:
bar_ex = alt.Chart(df_temp).mark_bar().encode(
    y = 'variable',
    x = 'mean(value)'
)
bar_ex

alt.Chart(...)

In [227]:
print(bar_ex.to_json())

{
  "$schema": "https://vega.github.io/schema/vega-lite/v5.8.0.json",
  "config": {
    "view": {
      "continuousHeight": 300,
      "continuousWidth": 300
    }
  },
  "data": {
    "name": "data-d2e74c2d023302ae426d408aaf7d2b7f"
  },
  "datasets": {
    "data-d2e74c2d023302ae426d408aaf7d2b7f": [
      {
        "value": 1,
        "variable": "a"
      },
      {
        "value": 2,
        "variable": "a"
      },
      {
        "value": 0,
        "variable": "a"
      },
      {
        "value": 2,
        "variable": "a"
      },
      {
        "value": 3,
        "variable": "b"
      },
      {
        "value": 2,
        "variable": "b"
      },
      {
        "value": 1,
        "variable": "b"
      },
      {
        "value": 4,
        "variable": "b"
      }
    ]
  },
  "encoding": {
    "x": {
      "aggregate": "mean",
      "field": "value",
      "type": "quantitative"
    },
    "y": {
      "field": "variable",
      "type": "nominal"
    }
  },
  "mark": {
  

## All data still included, even with a filter applied

One limitation of Altair is that **all of the data is included in the Vega-Lite JSON, even if you're using Altair to filter down to a subset, or not using part of it!**

Here we'll filter down to just a single value of data. *Note that `datum` is just the name for a single piece of data.*

In [228]:
from altair import datum

In [229]:
one_bar_ex = alt.Chart(df_temp).mark_bar().encode(
    y = 'variable',
    x = 'mean(value)'
).transform_filter(
    datum.variable == 'b'
)

one_bar_ex

alt.Chart(...)

...but all the data is still included in the JSON

In [230]:
one_bar_ex.to_dict()

{'config': {'view': {'continuousWidth': 300, 'continuousHeight': 300}},
 'data': {'name': 'data-d2e74c2d023302ae426d408aaf7d2b7f'},
 'mark': {'type': 'bar'},
 'encoding': {'x': {'aggregate': 'mean',
   'field': 'value',
   'type': 'quantitative'},
  'y': {'field': 'variable', 'type': 'nominal'}},
 'transform': [{'filter': "(datum.variable === 'b')"}],
 '$schema': 'https://vega.github.io/schema/vega-lite/v5.8.0.json',
 'datasets': {'data-d2e74c2d023302ae426d408aaf7d2b7f': [{'variable': 'a',
    'value': 1},
   {'variable': 'a', 'value': 2},
   {'variable': 'a', 'value': 0},
   {'variable': 'a', 'value': 2},
   {'variable': 'b', 'value': 3},
   {'variable': 'b', 'value': 2},
   {'variable': 'b', 'value': 1},
   {'variable': 'b', 'value': 4}]}}

---

# Saving to HTML files

Saving an Altair visualization to an HTML file is very easy! As we'll see below, you just have to chain a `.save('filename.html')` command on to the end of the specification.

## *ASIDE: complication*

**Unfortunately, there is one complication – if you've set the `alt.data_transformers.enable('json')` to avoid the MaxRowsError, you might want to turn that off before you save, so you can easily double-click to view your HTML file.** Or, you must at least understand that you'll need to run/have a web server to view your HTML visualizations

## JSON Data Transformer – effect on saved files

If you have set `alt.data_transformers.enable('json')` to avoid the MaxRowsError, Altair will automatically, behind the scenes, saves your data to a JSON file on your local filesystem (hard drive), and just reference that file name instead of embedding teh data in the JSON specification of the Vega-Lite visualization.

When you save an HTML file from Altair, the JSON specification it saves along with the file is exactly like it was in your Jupyter Notebook. If you have enabled the 'json' data transformer, the HTML file will reference the same JSON file URL for your data instead of embedding all the data in the JSON (and thus the HTML file).

The problem comes when you try to double-click on the HTML file from your hard drive to view it. 

- If the data is embedded, there is no problem – you will be able to see your static *or interactive* Vega-Lite chart that Altair has generated.
- **If the Vega-Lite chart refers to the data through a local file (URL), the page won't display properly when you double-click on it! Grabbing a local file is considered a [CORS request](https://en.wikipedia.org/wiki/Cross-origin_resource_sharing) in this scenario, and so *for security reasons* it isn't executed.**

### HTML files referring to local JSON files need a server

What you have to do to view any HTML-embedded Vega-Lite visualization that refers to a local JSON file is to put it (and the JSON data file) on a web server and view it through your browser. If you don't have easy access to a web server, you can run a temporary one locally by going into the directory with the files in a terminal on the Mac, or in the Anaconda prompt on Windows, and type:

`python -m http.server`

That should print out a message saying:

`Serving HTTP on 0.0.0.0 port 8000 (http://0.0.0.0:8000/) …`

which means you can go to http://0.0.0.0:8000/ in your browser and see the current directory. Click on the HTML file you want to view and the visualization should work fine.


## Changing back to default data transformer before saving HTML

**If you want to be able to just double-click your HTML file and view your static or interactive visualization, your data will need to be embedded directly in the HTML file.** To do this, you'll need to switch the data transformer back to 'default' before saving.

*Note: if you want to see the available transformers, you can use the `.names()` method. You can also write your own transformers, say to [save the JSON files to a sub-directory](https://altair-viz.github.io/user_guide/data_transformers.html#storing-json-data-in-a-separate-directory).*

In [231]:
alt.data_transformers.names()

['csv', 'default', 'json']

In [232]:
alt.data_transformers.enable('default')

DataTransformerRegistry.enable('default')

---

## Saving to HTML and JSON files

Documentation: [Saving Altair charts](https://altair-viz.github.io/user_guide/saving_charts.html)

Now, it's easy to save out an HTML file for the visualization, or the JSON specification. **These will get saved in the same directory as JupyterLab is running.**

In [233]:
bars = alt.Chart(df).mark_bar().encode(
    x = 'mean(response):Q',
    y = 'veg:N',
    color = 'trait:N'
)

bars.save('stacked_bars.html')
bars.save('stacked_bars.json')

## SVG and PNG require additional installs

Documentation: [Saving as PNG, SVG, and PDF](https://altair-viz.github.io/user_guide/saving_charts.html#png-svg-and-pdf-format)

**You can always render your visualization in the notebook and use the circular `...` button next to the visualization to render to SVG or PNG.** This method uses the web browswer you're using for your notebook to render to the file. You can also do the same from the HTML you generated in the previous step.

From the docs, "Saving these images requires an additional extension to run the javascript code necessary to interpret the Vega-Lite specification and output it in the form of an image. There are two packages that can be used to enable image export: 
[vl-convert](https://github.com/vega/vl-convert) or 
[altair_saver](http://github.com/altair-viz/altair_saver/)."

**What you want for now is vl-convert**, which can save to PNG and SVG. altair-saver can save to PDF, but requires extra dependencies and right now is not compatible with Altair 5.

```
pip install vl-convert-python
```

*If you changed the cell below to a Code cell and executed these, they would probably give you an error on these machines.*

---

# *EXTRAS*

## Tooltip and stacking order

In the original stacked bar, the color stacking puts the first legend item furthest left. If we wanted to change that we could **add an "order"**. Also, a **tooltip** is easy to add.

In [234]:
alt.Chart(df).mark_bar().encode(
    x = 'mean(response):Q',
    y = 'veg:N',
    color = 'trait:N',
    order = alt.Order('trait:N', sort='descending'),
    tooltip=['veg','trait','mean(response)']
).properties(height=150)

alt.Chart(...)

## *EXTRA:* changing the color scheme

The stacked bar chart you created in the first exercise re-used the same colors for the items that we've been using for the categoryies. 

**It's not a great idea to have the same color meaning two different things in nearby visualizations. We can switch the set of colors Altair uses by specifying a `Scale()` as an extra argument to `Color()`.**

*See the [Vega documentation](https://vega.github.io/vega/docs/schemes/) for a list of available color schemes*

In [239]:
alt.Chart(df).mark_bar().encode(
    x = 'trait:N',
    y = 'mean(response):Q',
    color = alt.Color('veg:N').scale(alt.Scale(scheme='set2'))
).properties(width=150)

alt.Chart(...)

In [236]:
alt.value('lightgray')

{'value': 'lightgray'}